In [1]:
# 🧱 PART 1: Imports and Setup
import requests
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
from tqdm import tqdm
import time

# Headers to simulate browser for NSE site
headers = {
    "User-Agent": "Mozilla/5.0"
}
#added in colab

In [2]:
# from nsepython import nse_top_gainers

# gainers = nse_top_gainers()
# print(gainers[:5])


In [3]:
import requests
import pandas as pd

headers = {
    "User-Agent": "Mozilla/5.0"
}

# NSE API URL (Live Variation)
url = "https://www.nseindia.com/api/live-analysis-variations?index=gainers"

# Create session for cookies
session = requests.Session()
session.headers.update(headers)
session.get("https://www.nseindia.com")  # initial hit for cookies

# Get JSON response
res = session.get(url)
data = res.json()

# Check available categories
available_categories = [cat[0] for cat in data['legends']]
print("Available Categories:", available_categories)

# Let's focus on NIFTY only for now
nifty_data = data.get("NIFTY", {}).get("data", [])

# Convert to DataFrame
df = pd.DataFrame(nifty_data)

# Rename and sort
df = df.rename(columns={
    "symbol": "Symbol",
    "ltp": "Last Price",
    "prev_price": "Previous Price",
    "perChange": "% Change",
    "trade_quantity": "Volume",
    "turnover": "Turnover (Cr)"
})

df = df.sort_values(by="% Change", ascending=False).reset_index(drop=True)

# Keep top 50 if available
df_top50 = df.head(50)

# Save to CSV
df_top50.to_csv("nifty_top_50_gainers.csv", index=False)

print("✅ Top 50 NIFTY gainers saved to 'nifty_top_50_gainers.csv'")


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [6]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
from tqdm import tqdm

# Load NIFTY 500 stock list
df_nifty500 = pd.read_csv("https://archives.nseindia.com/content/indices/ind_nifty500list.csv")
df_nifty500["Symbol_NS"] = df_nifty500["Symbol"] + ".NS"
tickers = df_nifty500["Symbol_NS"].tolist()

# Set date range
end = datetime.today()
start = end - timedelta(days=180)

top_stocks = []

for ticker in tqdm(tickers[:50]):  # Use [:50] for speed in testing
    try:
        data = yf.download(ticker, start=start, end=end, progress=False)
        if not data.empty:
            pct_change = ((data["Close"].iloc[-1] - data["Close"].iloc[0]) / data["Close"].iloc[0]) * 100
            top_stocks.append((ticker.replace(".NS", ""), pct_change))
        else:
            print(f"⚠️ No data for {ticker}")
    except Exception as e:
        print(f"⚠️ Error for {ticker}: {e}")

# Sort and save top gainers
df_top = pd.DataFrame(top_stocks, columns=["Symbol", "% Gain"])
df_top = df_top.sort_values(by="% Gain", ascending=False).reset_index(drop=True)
df_top.to_csv("top_50_gainers_last_6_months.csv", index=False)

print("✅ Done. Saved to top_50_gainers_last_6_months.csv")
df_top.head()


  0%|          | 0/50 [00:00<?, ?it/s]/tmp/ipython-input-6-548307409.py:19: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start, end=end, progress=False)
  2%|▏         | 1/50 [00:00<00:05,  8.93it/s]/tmp/ipython-input-6-548307409.py:19: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start, end=end, progress=False)
  4%|▍         | 2/50 [00:00<00:05,  8.56it/s]/tmp/ipython-input-6-548307409.py:19: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start, end=end, progress=False)
  6%|▌         | 3/50 [00:00<00:06,  7.64it/s]/tmp/ipython-input-6-548307409.py:19: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start, end=end, progress=False)
  8%|▊         | 4/50 [00:00<00:06,  7.61it/s]/tmp/ipython-input-6-548307409.py:19: FutureWa

ValueError: Can only compare identically-labeled Series objects

In [7]:
import yfinance as yf
from datetime import datetime, timedelta

ticker = "INFY.NS"
start = datetime.today() - timedelta(days=180)
end = datetime.today()

data = yf.download(ticker, start=start, end=end)
print(data.head())


/tmp/ipython-input-7-1570985513.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start, end=end)
[*********************100%***********************]  1 of 1 completed

Price             Close         High          Low         Open   Volume
Ticker          INFY.NS      INFY.NS      INFY.NS      INFY.NS  INFY.NS
Date                                                                   
2024-12-30  1879.554443  1889.415694  1860.325004  1889.119809  7789055
2024-12-31  1853.915161  1870.679288  1819.450138  1866.044548  3613063
2025-01-01  1856.380493  1866.685452  1847.998430  1847.998430  1838368
2025-01-02  1930.685059  1935.418507  1859.141753  1860.818117  7083645
2025-01-03  1911.850098  1925.853026  1895.332502  1925.853026  6218003


In [9]:
for ticker in tqdm(tickers[:10]):  # Start with 10 for quick test
    try:
        print(f"Downloading: {ticker}")
        data = yf.download(ticker, start=start, end=end, progress=False, auto_adjust=True)
        print(f"  Rows: {len(data)}")
        if not data.empty:
            pct_change = ((data["Close"].iloc[-1] - data["Close"].iloc[0]) / data["Close"].iloc[0]) * 100
            top_stocks.append((ticker.replace(".NS", ""), pct_change))
        else:
            print(f"⚠️ No data for {ticker}")
    except Exception as e:
        print(f"⚠️ Error for {ticker}: {e}")


 40%|████      | 4/10 [00:00<00:00, 38.80it/s]

Downloading: 360ONE.NS
  Rows: 122
Downloading: 3MINDIA.NS
  Rows: 122
Downloading: ABB.NS
  Rows: 122
Downloading: ACC.NS
  Rows: 122
Downloading: ACMESOLAR.NS
  Rows: 122
Downloading: AIAENG.NS
  Rows: 122
Downloading: APLAPOLLO.NS
  Rows: 122
Downloading: AUBANK.NS
  Rows: 122
Downloading: AWL.NS


100%|██████████| 10/10 [00:00<00:00, 41.43it/s]

  Rows: 122
Downloading: AADHARHFC.NS
  Rows: 122


In [13]:
display(pd.DataFrame(top_stocks))

,0,1
0,360ONE,Ticker 360ONE.NS -5.794767 dtype: float64
1,3MINDIA,Ticker 3MINDIA.NS -6.600859 dtype: float64
2,ABB,Ticker ABB.NS -10.430335 dtype: float64
3,ACC,Ticker ACC.NS -9.622572 dtype: float64
4,ACMESOLAR,Ticker ACMESOLAR.NS 7.108214 dtype: float64
...,...,...
65,AIAENG,Ticker AIAENG.NS -2.946669 dtype: float64
66,APLAPOLLO,Ticker APLAPOLLO.NS 18.007692 dtype: float64
67,AUBANK,Ticker AUBANK.NS 39.821547 dtype: float64
68,AWL,Ticker AWL.NS -21.064639 dtype: float64


In [ ]:
import pandas as pd

#url = "https://www1.nseindia.com/content/indices/ind_nifty500list.csv"
url = "https://archives.nseindia.com/content/indices/ind_nifty500list.csv"


df_nifty500 = pd.read_csv(url)
df_nifty500["Symbol_NS"] = df_nifty500["Symbol"] + ".NS"

# Save for later use
df_nifty500.to_csv("nifty_500_symbols.csv", index=False)
print("✅ NIFTY 500 symbols saved to nifty_500_symbols.csv")


✅ NIFTY 500 symbols saved to nifty_500_symbols.csv


In [ ]:
import yfinance as yf
from datetime import datetime, timedelta
from tqdm import tqdm
import time

start = datetime.today() - timedelta(days=365)
end = datetime.today()

all_data = []
missed_symbols=[]
for symbol in tqdm(df_nifty500['Symbol_NS'].tolist()):
    try:
        data = yf.download(symbol, start=start, end=end, progress=False, auto_adjust=False)
        data['symbol'] = symbol.replace('.NS', '')
        data.reset_index(inplace=True)
        all_data.append(data)
        time.sleep(0.5)  # avoid throttling
    except Exception as e:
        print(f"❌ {symbol}: {e}")
        missed_symbol.append(symbol)


 11%|████▌                                     | 54/503 [01:07<07:19,  1.02it/s]
1 Failed download:
['DMART.NS']: Timeout('Failed to perform, curl: (28) Connection timed out after 10000 milliseconds. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')
 29%|███████████▋                             | 144/503 [03:19<06:38,  1.11s/it]HTTP Error 404: 

1 Failed download:
['DUMMYRAYMN.NS']: YFTzMissingError('possibly delisted; no timezone found')
 52%|█████████████████████▏                   | 260/503 [06:28<09:33,  2.36s/it]Failed to get ticker 'JPPOWER.NS' reason: Failed to perform, curl: (28) Operation timed out after 10000 milliseconds with 0 bytes received. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.

1 Failed download:
['JPPOWER.NS']: YFTzMissingError('possibly delisted; no timezone found')
 54%|██████████████████████                   | 271/503 [07:12<16:07,  4.17s/it]Failed to get ticker 'JYOTHYLAB.NS' reason: Failed to perform, c